# Groundtruth handling

This script is used to create the similar/ and unique/ subcorpus of the lipade_groundtruth corpus.

In [28]:
import sys
path = "../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [29]:
from absolute_path import absolutePath
import pandas as pd
import os

In [ ]:
!ls $absolutePath/corpus/lipade_groundtruth

In [ ]:
!mv $absolutePath/corpus/lipade_groundtruth/donnees_IS_corrige.xlsx $absolutePath/corpus/lipade_groundtruth/groundtruth.xlsx

In [ ]:
!mv $absolutePath/corpus/lipade_groundtruth/ground_truth3.0/ $absolutePath/corpus/lipade_groundtruth/similar/

In [ ]:
!mv $absolutePath/corpus/lipade_groundtruth/historicaldataset/ $absolutePath/corpus/lipade_groundtruth/unique/

In [30]:
corpusPath = absolutePath + 'corpus/lipade_groundtruth/'

similarPath = os.path.join(corpusPath, 'similar/')
uniquePath  = os.path.join(corpusPath, 'unique/')

TODO : Correct manually all errors (add .tif and possibly copy paste the real name) in groundtruth.xlsx and reload the cells :

In [44]:
groundtruth = pd.read_excel(absolutePath + "/corpus/lipade_groundtruth/groundtruth.xlsx", header=None)

groups = [[image for image in group if type(image) is str] for group in groundtruth.values.tolist()]
for g in range(len(groups)):
    for i in range(len(groups[g])):
        if groups[g][i].endswith('.tif') or groups[g][i].endswith('.jpg'):
            groups[g][i] = os.path.join(similarPath, groups[g][i])
        else:
            groups[g][i] = os.path.join(similarPath, groups[g][i] + '.jpg')

In [45]:
for g in range(len(groups)):
    for i in range(len(groups[g])):
        if not os.path.exists(groups[g][i]):
            print(g,i,groups[g][i].split("/")[-1],"does not exist")

/ End of the TODO

In [35]:
for g in range(len(groups)):
    for i in range(len(groups[g])):
        if os.path.exists(os.path.join(uniquePath, groups[g][i].split("/")[-1])):
            os.remove(os.path.join(uniquePath, groups[g][i].split("/")[-1]))

# Metadata handling

This script is used to map the metadata of images of the corpus in an external disk with their associated local images.

In [1]:
import sys
path = "../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [2]:
from absolute_path import absolutePath
import pandas as pd
import os

In [111]:
distant_corpus = "/media/samuel-g/My Passport/STIS et FEI FEI/feifei/corpus_lipade/"
corpus = absolutePath + "corpus/lipade_groundtruth/"
xml_corpus = corpus + "xml/"

In [4]:
groundtruth = pd.read_excel(corpus + "groundtruth.xlsx", header=None)

groups = [[image for image in group if type(image) is str] for group in groundtruth.values.tolist()]
for g in range(len(groups)):
    for i in range(len(groups[g])):
        if groups[g][i].endswith('.tif') or groups[g][i].endswith('.jpg'):
            groups[g][i] = os.path.join(groups[g][i])
        else:
            groups[g][i] = os.path.join(groups[g][i] + '.jpg')

Similar images

In [72]:
metadata = {}

for img in os.listdir(corpus + "unique"):
    metadata[img] = {}

for img in os.listdir(corpus + "similar"):
    metadata[img] = {}

Agence Rol

In [112]:
imgPath = distant_corpus + "agence_rol/photos/jpg/"
metaPath = distant_corpus + "agence_rol/metadonnees/"

for img in os.listdir(corpus + "unique"):
    if img in os.listdir(imgPath):
        meta = img.split("_")[0] + ".xml"
        if meta in os.listdir(metaPath):
            metadata[img]["origin"] = "rol"
            xml = "rol/" + meta
            metadata[img]["xml"] = xml
            os.system(f'cp "{metaPath + meta}" "{xml_corpus + xml}"')

for img in os.listdir(corpus + "similar"):
    if img in os.listdir(imgPath):
        meta = img.split("_")[0] + ".xml"
        if meta in os.listdir(metaPath):
            metadata[img]["origin"] = "rol"
            xml = "rol/" + meta
            metadata[img]["xml"] = xml
            os.system(f'cp "{metaPath + meta}" "{xml_corpus + xml}"')

Cherau

In [74]:
imgPath = distant_corpus + "cherau/photos/jpg/"

for img in os.listdir(corpus + "unique"):
    if img in os.listdir(imgPath):
        metadata[img]["name"] = img
        metadata[img]["origin"] = "cherau"

for img in os.listdir(corpus + "similar"):
    if img in os.listdir(imgPath):
        metadata[img]["name"] = img
        metadata[img]["origin"] = "cherau"

Fonds Forbin

In [113]:
imgPath = distant_corpus + "fonds_forbin/"
suffix = "/photos/jpg/"
dirs = os.listdir(imgPath)

for img in os.listdir(corpus + "unique"):
    for dir in dirs:
        if img in os.listdir(imgPath + dir + suffix):
            metadata[img]["origin"] = "forbin"
            for file in os.listdir(imgPath + dir + "/"):
                if file[-4:] == ".xml":
                    xml = "forbin/" + file
                    metadata[img]["xml"] = xml
                    os.system(f'cp "{metaPath + meta}" "{xml_corpus + xml}"')
                if file[-5:] == ".xlsx":
                    tabular = pd.read_excel(imgPath + dir + "/" + file)
                    if 'num page' in tabular and 'num photo' in tabular:
                        nPage = int(img.split("_")[-2])
                        nPhoto = int(img.split("_")[-1].split(".")[0])
                        if len(tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)]) != 0:
                            line = tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)].iloc[0]
                            metadata[img]["df"] = line
            break

for img in os.listdir(corpus + "similar"):
    for dir in dirs:
        if img in os.listdir(imgPath + dir + suffix):
            metadata[img]["origin"] = "forbin"
            for file in os.listdir(imgPath + dir + "/"):
                if file[-4:] == ".xml":
                    xml = "forbin/" + file
                    metadata[img]["xml"] = xml
                    os.system(f'cp "{metaPath + meta}" "{xml_corpus + xml}"')
                if file[-5:] == ".xlsx":
                    tabular = pd.read_excel(imgPath + dir + "/" + file)
                    if 'num page' in tabular and 'num photo' in tabular:
                        nPage = int(img.split("_")[-2])
                        nPhoto = int(img.split("_")[-1].split(".")[0])
                        if len(tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)]) != 0:
                            line = tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)].iloc[0]
                            metadata[img]["df"] = line
            break

Presse

In [76]:
imgPath = distant_corpus + "presse/photos/jpg/"

for img in os.listdir(corpus + "unique"):
    if img in os.listdir(imgPath):
        metadata[img]["name"] = img
        metadata[img]["origin"] = "presse"

for img in os.listdir(corpus + "similar"):
    if img in os.listdir(imgPath):
        metadata[img]["name"] = img
        metadata[img]["origin"] = "presse"

Guerre illustree

In [114]:
imgPath = distant_corpus + "guerreillustree/"
suffix = "/photos/jpg/"
dirs = os.listdir(imgPath)

for img in os.listdir(corpus + "unique"):
    for dir in dirs:
        if img in os.listdir(imgPath + dir + suffix):
            metadata[img]["origin"] = "guerre illustree"
            metadata[img]["date"] = dir
            for file in os.listdir(imgPath + dir + "/"):
                if file[-4:] == ".xml":
                    xml = "guerreillustree/" + file
                    metadata[img]["xml"] = xml
                    os.system(f'cp "{metaPath + meta}" "{xml_corpus + xml}"')
                if file[-5:] == ".xlsx":
                    tabular = pd.read_excel(imgPath + dir + "/" + file)
                    if 'num page' in tabular and 'num photo' in tabular:
                        nPage = int(img.split("_")[-2])
                        nPhoto = int(img.split("_")[-1].split(".")[0])
                        if len(tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)]) != 0:
                            line = tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)].iloc[0]
                            metadata[img]["df"] = line
            break

for img in os.listdir(corpus + "similar"):
    for dir in dirs:
        if img in os.listdir(imgPath + dir + suffix):
            metadata[img]["origin"] = "guerre illustree"
            metadata[img]["date"] = dir
            for file in os.listdir(imgPath + dir + "/"):
                if file[-4:] == ".xml":
                    xml = "guerreillustree/" + file
                    metadata[img]["xml"] = xml
                    os.system(f'cp "{metaPath + meta}" "{xml_corpus + xml}"')
                if file[-5:] == ".xlsx":
                    tabular = pd.read_excel(imgPath + dir + "/" + file)
                    if 'num page' in tabular and 'num photo' in tabular:
                        nPage = int(img.split("_")[-2])
                        nPhoto = int(img.split("_")[-1].split(".")[0])
                        if len(tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)]) != 0:
                            line = tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)].iloc[0]
                            metadata[img]["df"] = line
            break

Sur le vif

In [115]:
imgPath = distant_corpus + "surlevif/"
suffix = "/photos/jpg/"
dirs = os.listdir(imgPath)

for img in os.listdir(corpus + "unique"):
    for dir in dirs:
        if img in os.listdir(imgPath + dir + suffix):
            metadata[img]["origin"] = "sur le vif"
            metadata[img]["date"] = dir
            for file in os.listdir(imgPath + dir + "/"):
                if file[-4:] == ".xml":
                    xml = "surlevif/" + file
                    metadata[img]["xml"] = xml
                    os.system(f'cp "{metaPath + meta}" "{xml_corpus + xml}"')
                if file[-5:] == ".xlsx":
                    tabular = pd.read_excel(imgPath + dir + "/" + file)
                    if 'num page' in tabular and 'num photo' in tabular:
                        nPage = int(img.split("_")[-2])
                        nPhoto = int(img.split("_")[-1].split(".")[0])
                        if len(tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)]) != 0:
                            line = tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)].iloc[0]
                            metadata[img]["df"] = line
            break

for img in os.listdir(corpus + "similar"):
    for dir in dirs:
        if img in os.listdir(imgPath + dir + suffix):
            metadata[img]["origin"] = "sur le vif"
            metadata[img]["date"] = dir
            for file in os.listdir(imgPath + dir + "/"):
                if file[-4:] == ".xml":
                    xml = "surlevif/" + file
                    metadata[img]["xml"] = xml
                    os.system(f'cp "{metaPath + meta}" "{xml_corpus + xml}"')
                if file[-5:] == ".xlsx":
                    tabular = pd.read_excel(imgPath + dir + "/" + file)
                    if 'num page' in tabular and 'num photo' in tabular:
                        nPage = int(img.split("_")[-2])
                        nPhoto = int(img.split("_")[-1].split(".")[0])
                        if len(tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)]) != 0:
                            line = tabular.loc[(tabular['num page'] == nPage) & (tabular['num photo'] == nPhoto)].iloc[0]
                            metadata[img]["df"] = line
            break

In [116]:
caracs = [
    'name', 'origin', 'num_page', 'num_photo', 'description', 'legende', 'legende_2', 'type', 'dimensions', 'xml', 'date', 
]

df = pd.DataFrame(columns=caracs)

for image_name, data in metadata.items():
    ligne = {}
    ligne["name"] = image_name
    for carac, value in data.items():
        if carac == "df":
            for carac2, value2 in value.items():
                if value2 == "/":
                    continue
                key = carac2.lower().replace(" ", "_").replace("é", "e")
                if key == "legendes":
                    ligne["legende"] = value2
                elif key == "legende2":
                    ligne["legende_2"] = value2
                elif "unnamed" not in key:
                    ligne[key] = value2
        else:
            ligne[carac] = value
    df = pd.concat([df, pd.DataFrame([ligne])], ignore_index=True)

In [117]:
df

,name,origin,num_page,num_photo,description,legende,legende_2,type,dimensions,xml,date,conflit,sensible
0,pho_2K24791_03_02.jpg,forbin,3,2,NaN,Near Doiran \nCrossing an iced river\n\nForbin,NaN,NaN,"17,5x12",forbin/2K24791.xml,NaN,NaN,NaN
1,np_surlevif_19160401_05_05.jpg,sur le vif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,surlevif/19160401.xml,19160401,NaN,NaN
2,np_surlevif_19160923_05_02.jpg,sur le vif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,surlevif/19160923.xml,19160923,NaN,NaN
3,np_guerreillustree_191701_11_04.jpg,guerre illustree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,guerreillustree/191701.xml,191701,NaN,NaN
4,np_guerreillustree_191712_21_02.jpg,guerre illustree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,guerreillustree/191712.xml,191712,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6200,thepenascolajournal_19111122_P4_01.jpg,presse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6201,pho_2K47160_26_01.jpg,forbin,26,1,NaN,"Arabs who have been executed by the Italians, ...",NaN,NaN,NaN,forbin/2K147160.xml,NaN,NaN,NaN
6202,SMNSWA_111213_29_07.jpg,presse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6203,thechickashadailyexpress_19111113_P1_02.jpg,presse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
df.to_csv(corpus + "metadata.xlsx")